In [1]:
import pandas as pd
import json
import os
from datetime import date

# Collecting the data from Twitter

In [3]:
# Import the Twython class
from twython import Twython
import json

# Load credentials from json file
with open("twitter_credentials.json", "r") as file:
    creds = json.load(file)

# Instantiate an object
python_tweets = Twython(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'])


In [7]:
import pysad

In [8]:
import importlib
importlib.reload(pysad)

<module 'pysad' from '/home/benjamin/Documents/EPFL/Research/sad/sad_tweets/pysad.py'>

In [9]:
init_accounts = pysad.initial_accounts()

In [10]:
init_accounts.list()

['swiss_climate_regular',
 'swiss_climate_controversial',
 'swiss_immigration',
 'french_tech_lesechos']

In [11]:

    
######Choose a category##############    
category_name = 'swiss_climate_controversial'
#category_name = 'swiss_climate_regular'
#category_name = 'french_tech_lesechos'
#category_name = 'swiss_immigration'
#####################################

username_list = init_accounts.accounts(category_name)

# create the path to save the experiment indexed with a date
today = date.today()
date_string = today.strftime("%Y%m%d")
print("date string =", date_string)

#date_string = '191128'

data_path = category_name + '/' + date_string+ '/'
#get_tweets = python_tweets.get_user_timeline(screen_name = username,  
#                                           count = 200, include_rts = True)
if not os.path.isdir(category_name):
    os.mkdir(category_name)
    print('Path created:',category_name)
if not os.path.isdir(data_path):
    os.mkdir(data_path)
    print('Path created:',data_path)
else:
    for f in os.listdir(data_path):
        os.remove(os.path.join(data_path, f))
    print('Cleaned path',data_path)

date string = 20200129
Path created: swiss_climate_controversial/20200129/


In [12]:
min_mentions = 2 # minimal number of mentions of a user to be followed
max_day_old = 7 # number max of days in the past
exploration_depth = 2 # mention of mention of mention of ... up to exploration depth

In [13]:
total_user_list = pysad.collect_tweets(username_list, data_path, python_tweets, min_mentions=min_mentions,
               max_day_old=max_day_old, exploration_depth=exploration_depth)

  0%|          | 0/12 [00:00<?, ?it/s]

Threshold set to 2 mentions.

******* Processing users at 0-hop distance *******


  8%|▊         | 1/12 [00:10<01:50, 10.04s/it]

Twitter API returned error None for user francisrichard.


 17%|█▋        | 2/12 [00:20<01:40, 10.04s/it]

...

Twitter API returned error None for user templivs.


  0%|          | 0/32 [00:00<?, ?it/s]

users with empty tweet list or no mention: ['francisrichard', 'MazdaArtaxerxes', 'templivs']

******* Processing users at 1-hop distance *******


100%|██████████| 32/32 [00:30<00:00,  1.04it/s]

users with empty tweet list or no mention: []
Total number of users collected:
190 190


# Loading the saved data into an edge table

In [14]:
import glob

edge_df = pd.DataFrame()
for filename in glob.glob(data_path + '*_mentions' +'_t' +str(min_mentions)+ '.json'):
    new_edge_df = pd.read_json(filename)
    print('{} with {} tweets.'.format(filename,len(new_edge_df)))
    edge_df = edge_df.append(new_edge_df)
edge_df.reset_index(drop=True, inplace=True)

swiss_climate_controversial/20200129/TS_SCI_MAJIC12_mentions_t2.json with 32 tweets.
swiss_climate_controversial/20200129/TheSolariReport_mentions_t2.json with 36 tweets.
swiss_climate_controversial/20200129/RestitutorOrien_mentions_t2.json with 35 tweets.
swiss_climate_controversial/20200129/AitiDouze_mentions_t2.json with 94 tweets.
swiss_climate_controversial/20200129/cocktail2Funk_mentions_t2.json with 35 tweets.
swiss_climate_controversial/20200129/FridayForFuture_mentions_t2.json with 15 tweets.
swiss_climate_controversial/20200129/QAnonAustria1_mentions_t2.json with 27 tweets.
swiss_climate_controversial/20200129/MoadabJ_mentions_t2.json with 37 tweets.
swiss_climate_controversial/20200129/TaoualitAmar_mentions_t2.json with 3 tweets.
swiss_climate_controversial/20200129/EnModeMacaron_mentions_t2.json with 9 tweets.
swiss_climate_controversial/20200129/Conflits_FR_mentions_t2.json with 59 tweets.
swiss_climate_controversial/20200129/ITanerof_mentions_t2.json with 44 tweets.
swiss

In [15]:
# display edges with number of hashtags >1
#edge_df[edge_df['hashtags'].apply(lambda x : len(x.split()))>1]
edge_df[edge_df['hashtags'].apply(lambda x : len(x))>1]

,user,mention,weight,hashtags,date,urls,text
40,TheSolariReport,KLE1967,1,"[ImpeachmentTrial, TrumpsRemovalTrial]",[2020-01-22 21:50:45],[],[Alexander Hamilton has been the star of the #...
107,AitiDouze,AmarilloLeones,1,"[GiletsJaunes, LoiAntiCasseurs]",[2020-01-25 17:32:56],[],[Quand la contestation a gagné la rue avec les...
108,AitiDouze,AnonymeCitoyen,2,"[pompiers, PompiersEnColere, Macron, France, P...","[2020-01-28 14:05:12, 2020-01-25 11:02:10]",[],[C'est une honte !\nLes #pompiers matraqués pa...
109,AitiDouze,AnonymeCitoyen1,1,"[Paris, Macron]",[2020-01-23 22:59:22],[],[🔴 - Lors de la manifestation aux flambeaux à ...
116,AitiDouze,DavidxCuba,1,"[Bilderberg, FrancMacon, Freemasons]",[2020-01-26 22:16:36],[],[@DavidxCuba @__Carlota @PlataApaza @HoChiMinh...
...,...,...,...,...,...,...,...
1995,Chabadalala,sputnik_fr,1,"[Allemagne, RotAmSee, Stuttgart]",[2020-01-24 13:41:57],[],[Une fusillade en #Allemagne: une opération de...
2008,Emonet4,Genethique,1,"[Directsénat, PJLBioethique]",[2020-01-28 18:18:55],[],[#Directsénat #PJLBioethique le gouvernement s...
2047,HarryoFfm,ConflictsW,1,"[France, Paris]",[2020-01-28 19:46:53],[],[Protests by firefighters over pension reforms...
2052,HarryoFfm,Eintracht,2,"[SGE, SGERBL]","[2020-01-25 16:50:07, 2020-01-23 17:59:08]",[],[Ende. Aus. Finito. 😎\n––––––\n#SGE #SGERBL 2:...


In [16]:
DEGREE_MIN = 2 # Minimal number of connections in the graph

G = pysad.graph_from_edgeslist(edge_df,DEGREE_MIN)
G.name = category_name
G.date = date_string

Creating the graph from the edge list
Field "hashtags" of class <class 'list'> converted to json string
Field "date" of class <class 'list'> converted to json string
Field "urls" of class <class 'list'> converted to json string
Field "text" of class <class 'list'> converted to json string
Nb of nodes: 1815
Nb of nodes after removing nodes with degree strictly smaller than 2: 202
removed 0 isolated nodes.


## Community detection and save full graph

In [17]:
G,clusters = pysad.detect_communities(G)

Communities saved on the graph as node attributes.
Nb of partitions: 7


In [18]:
# Save the graph
import networkx as nx

graphname = '00graph'

graphfilename = data_path + graphname + '_t' + str(min_mentions) + '_md' + str(DEGREE_MIN) +'_graph.gexf'
nx.write_gexf(G,graphfilename)
#nx.write_yaml(G,graphfilename)
#nx.write_pajek(G,graphfilename)
#nx.node_link_data(G,graphfilename)
print('Wrote',graphfilename)

Wrote swiss_climate_controversial/20200129/00graph_t2_md2_graph.gexf


In [19]:
G.date

'20200129'

## Analyze clusters

In [20]:
import importlib
importlib.reload(pysad)

<module 'pysad' from '/home/benjamin/Documents/EPFL/Research/sad/sad_tweets/pysad.py'>

In [21]:
cluster_dfi = pysad.indicator_table(clusters)
cluster_dfi

,Community,nb_nodes,k_max,max_kcore,norm_kcore,in_diversity,in_activity,activism,hierarchy,hierarchy2
0,0,34,3,4,0.117647,1.852941,5.323529,9.864187,25.500000,0.264706
1,1,48,2,21,0.437500,1.333333,2.604167,3.472222,4.571429,0.083333
2,2,21,3,5,0.238095,1.571429,4.380952,6.884354,12.600000,0.428571
3,3,51,3,10,0.196078,1.627451,2.431373,3.956940,15.300000,0.176471
4,4,29,3,6,0.206897,1.551724,10.482759,16.266350,14.500000,0.310345
5,5,12,1,12,1.000000,0.916667,3.250000,2.979167,1.000000,0.083333
6,6,7,1,7,1.000000,0.857143,1.714286,1.469388,1.000000,0.142857


In [22]:
cluster_dfi.sort_values(by='activism',ascending=False)

,Community,nb_nodes,k_max,max_kcore,norm_kcore,in_diversity,in_activity,activism,hierarchy,hierarchy2
4,4,29,3,6,0.206897,1.551724,10.482759,16.266350,14.500000,0.310345
0,0,34,3,4,0.117647,1.852941,5.323529,9.864187,25.500000,0.264706
2,2,21,3,5,0.238095,1.571429,4.380952,6.884354,12.600000,0.428571
3,3,51,3,10,0.196078,1.627451,2.431373,3.956940,15.300000,0.176471
1,1,48,2,21,0.437500,1.333333,2.604167,3.472222,4.571429,0.083333
5,5,12,1,12,1.000000,0.916667,3.250000,2.979167,1.000000,0.083333
6,6,7,1,7,1.000000,0.857143,1.714286,1.469388,1.000000,0.142857


In [23]:
cluster_dfi.sort_values(by='in_diversity',ascending=False)

,Community,nb_nodes,k_max,max_kcore,norm_kcore,in_diversity,in_activity,activism,hierarchy,hierarchy2
0,0,34,3,4,0.117647,1.852941,5.323529,9.864187,25.500000,0.264706
3,3,51,3,10,0.196078,1.627451,2.431373,3.956940,15.300000,0.176471
2,2,21,3,5,0.238095,1.571429,4.380952,6.884354,12.600000,0.428571
4,4,29,3,6,0.206897,1.551724,10.482759,16.266350,14.500000,0.310345
1,1,48,2,21,0.437500,1.333333,2.604167,3.472222,4.571429,0.083333
5,5,12,1,12,1.000000,0.916667,3.250000,2.979167,1.000000,0.083333
6,6,7,1,7,1.000000,0.857143,1.714286,1.469388,1.000000,0.142857


In [24]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

# Hashtags, dates and urls
Hashtags, dates and urls are on the edges of the network.
We can get the most common hashtags within a community and also betwenn communities using the edges that connect them.

In [25]:
cluster_df = pysad.dates_tags_table(clusters)
cluster_df.iloc[:,:-2]

,Community,Average date,Deviation (days),hashtag0,hashtag1,hashtag2,hashtag3,hashtag4
0,0,2020-01-26,1,Klimawandel,Kohleausstieg,KeinKonsens,Windkraft,FridaysForFuture
1,1,2020-01-27,1,France,coronavirus,climat,pluie,chemtrails
2,2,2020-01-27,1,coronavirus,Chine,Wuhan,PompiersEnColère,pompiers
3,3,2020-01-28,1,pompiers,greve28janvier,Paris,GM2L,ASKPPG
4,4,2020-01-28,1,greve28janvier,pompiers,PompiersEnColere,PompiersEnColère,Pompiers
5,5,2020-01-28,0,Ravier2022,,,,
6,6,2020-01-27,1,5Giganetz,GeoEngineering,Chemtrails,Schweiz,CoronaVirus


In [26]:
community_table = cluster_dfi.merge(cluster_df)
community_table = community_table.iloc[:,:-2]
community_table

,Community,nb_nodes,k_max,max_kcore,norm_kcore,in_diversity,in_activity,activism,hierarchy,hierarchy2,Average date,Deviation (days),hashtag0,hashtag1,hashtag2,hashtag3,hashtag4
0,0,34,3,4,0.117647,1.852941,5.323529,9.864187,25.500000,0.264706,2020-01-26,1,Klimawandel,Kohleausstieg,KeinKonsens,Windkraft,FridaysForFuture
1,1,48,2,21,0.437500,1.333333,2.604167,3.472222,4.571429,0.083333,2020-01-27,1,France,coronavirus,climat,pluie,chemtrails
2,2,21,3,5,0.238095,1.571429,4.380952,6.884354,12.600000,0.428571,2020-01-27,1,coronavirus,Chine,Wuhan,PompiersEnColère,pompiers
3,3,51,3,10,0.196078,1.627451,2.431373,3.956940,15.300000,0.176471,2020-01-28,1,pompiers,greve28janvier,Paris,GM2L,ASKPPG
4,4,29,3,6,0.206897,1.551724,10.482759,16.266350,14.500000,0.310345,2020-01-28,1,greve28janvier,pompiers,PompiersEnColere,PompiersEnColère,Pompiers
5,5,12,1,12,1.000000,0.916667,3.250000,2.979167,1.000000,0.083333,2020-01-28,0,Ravier2022,,,,
6,6,7,1,7,1.000000,0.857143,1.714286,1.469388,1.000000,0.142857,2020-01-27,1,5Giganetz,GeoEngineering,Chemtrails,Schweiz,CoronaVirus


In [27]:
community_table.sort_values('activism',ascending=False).drop(['Average date','Deviation (days)'],axis=1)

,Community,nb_nodes,k_max,max_kcore,norm_kcore,in_diversity,in_activity,activism,hierarchy,hierarchy2,hashtag0,hashtag1,hashtag2,hashtag3,hashtag4
4,4,29,3,6,0.206897,1.551724,10.482759,16.266350,14.500000,0.310345,greve28janvier,pompiers,PompiersEnColere,PompiersEnColère,Pompiers
0,0,34,3,4,0.117647,1.852941,5.323529,9.864187,25.500000,0.264706,Klimawandel,Kohleausstieg,KeinKonsens,Windkraft,FridaysForFuture
2,2,21,3,5,0.238095,1.571429,4.380952,6.884354,12.600000,0.428571,coronavirus,Chine,Wuhan,PompiersEnColère,pompiers
3,3,51,3,10,0.196078,1.627451,2.431373,3.956940,15.300000,0.176471,pompiers,greve28janvier,Paris,GM2L,ASKPPG
1,1,48,2,21,0.437500,1.333333,2.604167,3.472222,4.571429,0.083333,France,coronavirus,climat,pluie,chemtrails
5,5,12,1,12,1.000000,0.916667,3.250000,2.979167,1.000000,0.083333,Ravier2022,,,,
6,6,7,1,7,1.000000,0.857143,1.714286,1.469388,1.000000,0.142857,5Giganetz,GeoEngineering,Chemtrails,Schweiz,CoronaVirus


In [28]:
community_table[['Community', 'nb_nodes', 'activism','hierarchy', 'hashtag0', 'hashtag1', 'hashtag2', 'hashtag3', 'hashtag4']].sort_values('activism',ascending=False)

,Community,nb_nodes,activism,hierarchy,hashtag0,hashtag1,hashtag2,hashtag3,hashtag4
4,4,29,16.266350,14.500000,greve28janvier,pompiers,PompiersEnColere,PompiersEnColère,Pompiers
0,0,34,9.864187,25.500000,Klimawandel,Kohleausstieg,KeinKonsens,Windkraft,FridaysForFuture
2,2,21,6.884354,12.600000,coronavirus,Chine,Wuhan,PompiersEnColère,pompiers
3,3,51,3.956940,15.300000,pompiers,greve28janvier,Paris,GM2L,ASKPPG
1,1,48,3.472222,4.571429,France,coronavirus,climat,pluie,chemtrails
5,5,12,2.979167,1.000000,Ravier2022,,,,
6,6,7,1.469388,1.000000,5Giganetz,GeoEngineering,Chemtrails,Schweiz,CoronaVirus


In [119]:
import importlib
importlib.reload(pysad)

<module 'pysad' from '/home/benjamin/Documents/EPFL/Research/sad/sad_tweets/pysad.py'>

In [97]:
clusters[0]=pysad.cluster_attributes(clusters[0])
table_dic = pysad.cluster_tables(clusters[0])

In [70]:
pysad.save_excel(table_dic,data_path + cluster_id + '_infos.xlsx')

Data saved to swiss_climate_controversial/20200129/cluster0_infos.xlsx


## Automatic processing of all clusters

In [120]:
for c_id in clusters:
    cgraph = clusters[c_id]
    cgraph = pysad.cluster_attributes(cgraph)
    table_dic = pysad.cluster_tables(cgraph)
    cluster_filename = data_path + 'cluster' + str(c_id)
    pysad.save_excel(table_dic,cluster_filename + '_infos.xlsx')
    pysad.save_graph(cgraph,cluster_filename + 'graph.gexf')

Data saved to swiss_climate_controversial/20200129/cluster0_infos.xlsx
Graph saved to swiss_climate_controversial/20200129/cluster0graph.gexf
Data saved to swiss_climate_controversial/20200129/cluster1_infos.xlsx
Graph saved to swiss_climate_controversial/20200129/cluster1graph.gexf
Data saved to swiss_climate_controversial/20200129/cluster2_infos.xlsx
Graph saved to swiss_climate_controversial/20200129/cluster2graph.gexf
Data saved to swiss_climate_controversial/20200129/cluster3_infos.xlsx
Graph saved to swiss_climate_controversial/20200129/cluster3graph.gexf
Data saved to swiss_climate_controversial/20200129/cluster4_infos.xlsx
Graph saved to swiss_climate_controversial/20200129/cluster4graph.gexf
Data saved to swiss_climate_controversial/20200129/cluster5_infos.xlsx
Graph saved to swiss_climate_controversial/20200129/cluster5graph.gexf
Data saved to swiss_climate_controversial/20200129/cluster6_infos.xlsx
Graph saved to swiss_climate_controversial/20200129/cluster6graph.gexf


## Process text of all tweets in a community 

In [60]:
import preprocessor as p

In [100]:
cx = clusters[2]
cx=pysad.cluster_attributes(cx)
table_dic = pysad.cluster_tables(cx)

In [101]:
text_table = table_dic['text'].copy()
for idx,row in table_dic['text'].iterrows():
    filtered_text = p.clean(row['text'])
    text_table.loc[idx,'filtered text'] = filtered_text
    

In [102]:
text_table

,text,filtered text,bcentrality
0,🇦🇺 [FLASH] - Les incendies en #Australie ont f...,🇦🇺 [FLASH] - Les incendies en ont favorisé la ...,0.144444
1,🦠 URGENT - Les scientifiques australiens ont r...,🦠 URGENT - Les scientifiques australiens ont r...,0.114286
2,"🔴 URGENT - Michael D'Andrea, #CIA, qui a joué ...","URGENT - Michael D'Andrea, , qui a joué un rôl...",0.114286
3,@Lila_Sydney @plusdecheveux @SashaTrib @Confli...,Elle a été publiée sur le twitter de,0.114286
4,🇫🇷 FLASH INFO - Un homme a été placé en quaran...,🇫🇷 FLASH INFO - Un homme a été placé en quaran...,0.114286
...,...,...,...
87,@RestitutorOrien Merci @RestitutorOrien pour l...,Merci pour la mise à jour des info,0.030952
88,Pékin a signalé dimanche 5 nouveaux cas confir...,Pékin a signalé dimanche nouveaux cas confirmé...,0.030952
89,@RestitutorOrien Petite précision pour l'atter...,Petite précision pour l'atterissage en : c'éta...,0.021429
90,🦠 Épidémie de #coronavirus : les Américains ra...,🦠 Épidémie de : les Américains rapatriés de la...,0.021429


In [87]:
from nltk import word_tokenize
from nltk.corpus import stopwords

In [86]:
from string import punctuation

In [103]:
fulltext = ''
for text in text_table['filtered text']:#.iterrows():
    fulltext += ' ' + text

In [104]:
fulltext

' 🇦🇺 [FLASH] - Les incendies en ont favorisé la prolifération de l\'araignée à toile d\'entonnoir. Son venin est extrêmement mortel pour l\'homme. Pour l\'heure, on recense aucunes morsures chez l\'homme. \U0001f9a0 URGENT - Les scientifiques australiens ont réussi à recréer le pour la ère fois. En , les scientifiques avaient reproduit le génome mais pas le en lui-même. Les médecins pourront utiliser cette copie pour tester des . (BBC) URGENT - Michael D\'Andrea, , qui a joué un rôle clef dans la mort de , a été tué dans le crash d\'hier d\'un en . Les talibans assurent avoir abattu l\'appareil, ce que les démentent. L\' évoque un souci technique. - Elle a été publiée sur le twitter de 🇫🇷 FLASH INFO - Un homme a été placé en quarantaine à car il est suspecté d\'être infecté par le . Le SAMU est intervenu proche du centre-ville. Des analyses infectiologiques sont actuellement menées par le , le est en état d\'alerte. (Paris Normandie) Si saute (certainement ce soir malheureusement) nous

In [83]:
text

'FYI, Andreas Kalcker - who is researching MMS/CDL for &gt;10 years - is currently in the process of approving CDL as a drug in Switzerland.'

In [105]:
stop_words = stopwords.words('french') + list(punctuation)
 
def tokenize(text):
    words = word_tokenize(text)
    words = [w.lower() for w in words]
    return [w for w in words if w not in stop_words and not w.isdigit()]
 

In [93]:
#import nltk
#nltk.download()

In [107]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/benjamin/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [108]:
tktext = tokenize(fulltext)

In [109]:
tktext

['🇦🇺',
 'flash',
 'incendies',
 'favorisé',
 'prolifération',
 "l'araignée",
 'toile',
 "d'entonnoir",
 'venin',
 'extrêmement',
 'mortel',
 "l'homme",
 "l'heure",
 'recense',
 'aucunes',
 'morsures',
 'chez',
 "l'homme",
 '\U0001f9a0',
 'urgent',
 'scientifiques',
 'australiens',
 'réussi',
 'recréer',
 'ère',
 'fois',
 'scientifiques',
 'reproduit',
 'génome',
 'lui-même',
 'médecins',
 'pourront',
 'utiliser',
 'cette',
 'copie',
 'tester',
 'bbc',
 'urgent',
 'michael',
 "d'andrea",
 'a',
 'joué',
 'rôle',
 'clef',
 'mort',
 'a',
 'tué',
 'crash',
 "d'hier",
 "d'un",
 'talibans',
 'assurent',
 'avoir',
 'abattu',
 "l'appareil",
 'démentent',
 'évoque',
 'souci',
 'technique',
 'a',
 'publiée',
 'twitter',
 '🇫🇷',
 'flash',
 'info',
 'homme',
 'a',
 'placé',
 'quarantaine',
 'car',
 'suspecté',
 "d'être",
 'infecté',
 'samu',
 'intervenu',
 'proche',
 'centre-ville',
 'analyses',
 'infectiologiques',
 'actuellement',
 'menées',
 'état',
 "d'alerte",
 'paris',
 'normandie',
 'si',
 's

In [110]:
# Calculate frequency distribution
fdist = nltk.FreqDist(tktext)

# Output top 50 words

for word, frequency in fdist.most_common(50):
    print(u'{};{}'.format(word, frequency))

a;25
cas;13
’;11
flash;10
🦠;8
...;8
🇫🇷;7
plus;7
personnes;7
virus;7
si;6
états-unis;6
c'est;6
pouvez;6
info;5
quarantaine;5
ici;5
``;5
'';5
bien;5
chinois;5
être;5
quelques;5
urgent;4
coronavirus;4
manifestation;4
image;4
elles;4
confirmés;4
«;4
»;4
jour;4
tweeté;4
place;4
ça;4
tweets;4
cette;3
twitter;3
infecté;3
tensions;3
entre;3
toujours;3
alerte;3
forces;3
l'ordre;3
tentent;3
toutes;3
épidémie;3
cdc;3
américain;3
